In [1]:
import numpy as np
import scipy
from scipy import spatial
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import time
from lmafit import lmafit_mc_adp
import json

In [2]:
INVALID = 0.0

In [3]:
def IOU(A, B):
    '''Intesection over Union'''
    return float(len(A & B)) / len(A | B)

def RMSE(x, y):
    assert(x.shape == y.shape)
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

def similarity_matrix(X, sim_measure):
    dist = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(X, sim_measure))
    dist[np.isnan(dist)] = 0.0
    return 1 - dist

def similarity_matrix_cosine(X):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :return: n x n cosine similarity matrix
    '''
    return similarity_matrix(X, 'cosine')

def similarity_matrix_knn(X, k, dim=2):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :return: n x n KNN similarity matrix
    '''
    n = X.shape[0]
    if k > n:
        raise IndexError('Only {} points, cannot have {} neighbors'
                         .format(n, k))
    sim = np.zeros((n, n))
    dist = spatial.distance_matrix(X, X, dim)
    for i in range(n):
        # ind is the indices of the closest k points to point i
        ind = np.argpartition(dist[i], -k)[:k]
        sim[i, ind] = 1.0 / k
    return sim

In [4]:
def get_rating(user_id, movie_id, ratings, sim):
    n = ratings.shape[0]
    sim_values = sim[user_id, :]
    movie_ratings = ratings[:, movie_id]
    
    # Use only valid ratings
    valid_ind = movie_ratings != INVALID
    sim_values = sim_values[valid_ind]
    movie_ratings = movie_ratings[valid_ind]
    pred_rating = sim_values.dot(movie_ratings)
    total = sim_values.sum()
    return pred_rating / total if total > 0 else pred_rating

In [5]:
def test_collaborative_filtering(ratings, p, sim_measure='cosine', k=5, dim=2):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param sim_measure: Similarity measure to use
    :param p: Fraction of data to use as test-set
    :param k: In case of kNN, value of k to use
    :param dim: In case of kNN, value of dim to use
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    test_subset = np.random.choice(np.arange(N), int(p * N), replace=False)
    test_ind = valid_ind[0][test_subset], valid_ind[1][test_subset]
    num_test = test_subset.shape[0]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID

    # Compute similarity matrix
    sim = similarity_matrix(train_ratings, sim_measure.lower())
    pred_ratings = np.zeros((num_test,))
    
    # Compute error
    true_ratings = ratings[test_ind]
    for i, (u, m) in enumerate(zip(*test_ind)):
        pred_ratings[i] = get_rating(u, m, train_ratings, sim)
    
    pred_ratings = pred_ratings.clip(1, 5)
    return RMSE(true_ratings, pred_ratings)


def test_matrix_completion(ratings, p, k):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set
    :param k: Estimated rank of matrix to use for LMaFit
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    num_test = int(p * N)
    test_ind = valid_ind[0][indices[:num_test]], valid_ind[1][indices[:num_test]]
    train_ind = valid_ind[0][indices[num_test:]], valid_ind[1][indices[num_test:]]
        
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Run LMaFit
    a, b, _ = lmafit_mc_adp(ratings.shape[0], ratings.shape[1], k, train_ind, ratings[train_ind], None)
    completed = a.dot(b)
    
    # Compute error
    completed = completed.clip(1, 5)
    return RMSE(ratings[test_ind], completed[test_ind])

## With Genres

In [6]:
import pandas as pd
import numpy as np
import csv

movie_file = "ml-1m/movies.dat"
m_names = ["bad_index", "Title", "Genre"]
movies = pd.read_csv(movie_file, nrows=1000000, header=None, names=m_names, sep="::", engine='python')
movie_to_index = dict((m,i) for i,m in zip(movies.index, movies["bad_index"]))
movie_to_genre = dict((i,g) for i,g in zip(movies.index, movies["Genre"]))

In [7]:
genre_to_int = {
    'Action' : 0,
    'Adventure' : 1,
    'Animation' : 2,
    'Children\'s' : 3,
    'Comedy' : 4,
    'Crime' : 5,
    'Documentary' : 6,
    'Drama' : 7,
    'Fantasy' : 8,
    'Film-Noir' : 9, 
    'Horror' : 10,
    'Musical' : 11,
    'Mystery' : 12,
    'Romance' : 13,
    'Sci-Fi' : 14,
    'Thriller' : 15,
    'War' : 16,
    'Western' : 17, 
}
NUM_GENRES = 18

def get_genres_for_movie(movie, curr):
    inds = [genre_to_int[genre] for genre in movie_to_genre[movie].split('|')]
    for i in inds:
        curr[i] += 1
    return curr

def get_top_k_genres_for_user(mat, user, k, return_prefs=False):
    movies = mat[user]
    genre_prefs = np.zeros(len(genre_to_int))
    average_rating = movies[np.nonzero(movies)].mean()
    for i, movie in enumerate(movies):
        if movie > average_rating:
            genre_prefs = get_genres_for_movie(i, genre_prefs)
    if return_prefs:
        return np.flip(np.sort(genre_prefs))[:k], np.flip(np.argsort(genre_prefs), axis=0)[:k]
    else:
        return np.flip(np.argsort(genre_prefs), axis=0)[:k]

In [8]:
def genre_similarity_matrix_by_overlap(X, top_k):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :param top_k: Top-k genres will be considered
    :return: n x n genre similarity matrix
    '''
    n = X.shape[0]  # Number of users
    genre_ratings = np.zeros((NUM_GENRES, n, n), dtype=np.bool)
    for u in range(n):
        best_genres = get_top_k_genres_for_user(X, u, top_k)
        genre_ratings[best_genres, u, :] = True

    # Now, genre_ratings[i, u, :] == True iff user u has genre i in their top-k genres
    # overlap_by_genre[i, u, v] == True iff users u and v both have genre i in their top-k genres
    overlap_by_genre = np.logical_and(genre_ratings, genre_ratings.transpose((0, 2, 1)))
    assert(overlap_by_genre.shape == (NUM_GENRES, n, n))

    return overlap_by_genre.sum(axis=0)

def matrix_completion_by_genres(X, top_k=3):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :param top_k: Top-k genres will be considered
    :return: n x p matrix, with predicted values for movie ratings for each user
    using info about genres a user likes
    '''
    completed = np.zeros(X.shape)
    
    num_users, num_movies = X.shape
    user_genres = [set(get_top_k_genres_for_user(X, u, k=top_k)) for u in range(num_users)]
    movie_genres = [{genre_to_int[g] for g in movie_to_genre[m].split('|')} for m in range(num_movies)]
    
    for u in range(num_users):
        for m in range(num_movies):
            completed[u, m] = 3.0 + 2.0 * IOU(user_genres[u], movie_genres[m])
    return completed

# def rate_movies_by_user_history(X, indices, top_k=3):
#     '''
#     :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
#     :param top_k: Top-k genres will be considered
#     :return: Returns overlap values for users and movies provided in indices
#     '''
    
#     num_users, num_movies = X.shape
#     user_genres = [set(get_top_k_genres_for_user(X, u, k=top_k)) for u in range(num_users)]
#     movie_genres = [{genre_to_int[g] for g in movie_to_genre[m].split('|')} for m in range(num_movies)]
#     overlap = []
#     for u, m in zip(indices[0], indices[1]):
#         overlap.append(IOU(user_genres[u], movie_genres[m]))
    
#     return overlap

In [9]:
def test_genre_based_matrix_completion(ratings, p, top_k=3):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    num_test = int(p * N)
    test_ind = valid_ind[0][indices[:num_test]], valid_ind[1][indices[:num_test]]
    train_ind = valid_ind[0][indices[num_test:]], valid_ind[1][indices[num_test:]]
        
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Run LMaFit
    completed = matrix_completion_by_genres(X=ratings, top_k=top_k)
    
    # Compute error
    completed = completed.clip(1, 5)
    return RMSE(ratings[test_ind], completed[test_ind])

In [10]:
def test_collaborative_filtering_with_genre_bias(ratings, p, bias, sim_measure='cosine', top_k=3):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param sim_measure: Similarity measure to use
    :param p: Fraction of data to use as test-set
    :param top_k: Top-k genres will be considered
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    num_test = int(p * N)
    test_ind = valid_ind[0][indices[:num_test]], valid_ind[1][indices[:num_test]]
    train_ind = valid_ind[0][indices[num_test:]], valid_ind[1][indices[num_test:]]
        
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Compute similarity matrix
    sim = similarity_matrix(train_ratings, sim_measure.lower())
    
    # Run content based matrix completion
#     overlap = rate_movies_by_user_history(train_ratings, indices=test_ind, top_k=top_k)

    pred_ratings = np.zeros((num_test,))
    
    # Compute error
    true_ratings = ratings[test_ind]
    for i, (u, m) in enumerate(zip(*test_ind)):
        pred_ratings[i] = get_rating(u, m, train_ratings, sim)
        
        user_genres = set(get_top_k_genres_for_user(train_ratings, u, k=top_k))
        movie_genres = {genre_to_int[g] for g in movie_to_genre[m].split('|')}
        if len(user_genres & movie_genres) > 0:
            pred_ratings[i] += bias
        else:
            pred_ratings[i] -= bias
    
    pred_ratings = pred_ratings.clip(1, 5)
    return RMSE(true_ratings, pred_ratings)

In [11]:
def test_collaborative_filtering_with_genre_similarity(ratings, p, bias, sim_measure='cosine', top_k=3):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param sim_measure: Similarity measure to use
    :param p: Fraction of data to use as test-set
    :param top_k: Top-k genres will be considered
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    test_subset = np.random.choice(np.arange(N), int(p * N), replace=False)
    test_ind = valid_ind[0][test_subset], valid_ind[1][test_subset]
    num_test = test_subset.shape[0]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Compute the genre similarity matrix
    genre_sim = genre_similarity_matrix_by_overlap(train_ratings, top_k=top_k)
    
    # Compute similarity matrix
    sim = similarity_matrix(train_ratings, sim_measure.lower())

    # Combine the regular similarity and the genre similarity
    sim += bias * genre_sim
    
    pred_ratings = np.zeros((num_test,))
    
    # Compute error
    true_ratings = ratings[test_ind]
    for i, (u, m) in enumerate(zip(*test_ind)):
        pred_ratings[i] = get_rating(u, m, train_ratings, sim)
    
    pred_ratings = pred_ratings.clip(1, 5)
    return RMSE(true_ratings, pred_ratings)

## Measurements

In [12]:
true_ratings = np.load('1M_ratings_np.npy')

In [13]:
p_values = [0.2, 0.4, 0.6, 0.8]

In [14]:
sim_measures = ['cosine', 'hamming']

In [16]:
regular_errors = {s: {} for s in sim_measures}
for sim_measure in sim_measures:
    for p in p_values:
        err = test_collaborative_filtering(ratings=true_ratings, p=p, sim_measure=sim_measure)
        regular_errors[sim_measure][p] = err

print(regular_errors)

{'cosine': {0.8: 0.98926427214983281, 0.6: 0.97654606173502456, 0.4: 0.97513725400336693, 0.2: 0.97344218358250245}, 'hamming': {0.8: 0.98834014694141203, 0.6: 0.9824942782005992, 0.4: 0.97944235822234826, 0.2: 0.97906590044665009}}


In [17]:
# knn_errors_by_k = {}
# for k in [5, 10, 50, 100, 500, 1000]:
#     err = test_collaborative_filtering(ratings=true_ratings, p=0.5, sim_measure='knn', k=k)
#     knn_errors_by_k[k] = err

# print(sorted(knn_errors_by_k.items(), key=lambda x: x[0]))

### With Genres

In [18]:
bias_values = [0.0, 0.1, 0.2, 0.5, 1.0]

In [ ]:
genre_sim_errors = {s: {p: {} for p in p_values} for s in sim_measures}
for sim_measure in sim_measures:
    for p in p_values:
        for bias in bias_values:
            err = test_collaborative_filtering_with_genre_similarity(true_ratings, p=p, bias=bias, 
                                                                     sim_measure=sim_measure, top_k=3)
            genre_sim_errors[sim_measure][p][bias] = err
            print(sim_measure, p, bias, err)

print(genre_sim_errors)

('cosine', 0.2, 0.0, 0.97424677714293983)
('cosine', 0.2, 0.1, 0.97528379695451017)
('cosine', 0.2, 0.2, 0.97586986886153404)
('cosine', 0.2, 0.5, 0.97395946019486723)
('cosine', 0.2, 1.0, 0.97573981350137895)
('cosine', 0.4, 0.0, 0.97502402984374115)
('cosine', 0.4, 0.1, 0.97567241474729272)
('cosine', 0.4, 0.2, 0.97664830674056802)
('cosine', 0.4, 0.5, 0.97583035448977051)
('cosine', 0.4, 1.0, 0.97722370567660821)
('cosine', 0.6, 0.0, 0.97690866143266852)
('cosine', 0.6, 0.1, 0.97809897161149428)
('cosine', 0.6, 0.2, 0.98023276137296322)
('cosine', 0.6, 0.5, 0.97842288848229841)
('cosine', 0.6, 1.0, 0.97964691547759775)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


('cosine', 0.8, 0.0, 0.98907491599525221)
('cosine', 0.8, 0.1, 0.98551005983145734)
('cosine', 0.8, 0.2, 0.98596535251948858)
('cosine', 0.8, 0.5, 0.98633699060636137)
('cosine', 0.8, 1.0, 0.9870132275275596)
('hamming', 0.2, 0.0, 0.97807946757195763)
('hamming', 0.2, 0.1, 0.98089461093263264)
('hamming', 0.2, 0.2, 0.97774153743158299)
('hamming', 0.2, 0.5, 0.97674293944893231)
('hamming', 0.2, 1.0, 0.97545973243304529)
('hamming', 0.4, 0.0, 0.97931573170801023)
('hamming', 0.4, 0.1, 0.97980361751011036)
('hamming', 0.4, 0.2, 0.97940904442174903)
('hamming', 0.4, 0.5, 0.97959297262592959)
('hamming', 0.4, 1.0, 0.97880131895315114)
('hamming', 0.6, 0.0, 0.98168463490313251)
('hamming', 0.6, 0.1, 0.98171775584710774)
('hamming', 0.6, 0.2, 0.98212523619453251)
('hamming', 0.6, 0.5, 0.98045052470104177)
('hamming', 0.6, 1.0, 0.98116246024004983)
('hamming', 0.8, 0.0, 0.98791330822347079)
('hamming', 0.8, 0.1, 0.9880916744826943)
('hamming', 0.8, 0.2, 0.98704649999775818)
('hamming', 0.8, 0

In [ ]:
genre_bias_errors = {s: {p: {} for p in p_values} for s in sim_measures}
for sim_measure in sim_measures:
    for p in p_values:
        for bias in bias_values:
            err = test_collaborative_filtering_with_genre_bias(true_ratings, p=p, bias=bias, 
                                                               sim_measure=sim_measure, top_k=3)
            genre_bias_errors[sim_measure][p][bias] = err
            print(sim_measure, p, bias, err)

print(genre_bias_errors)

('cosine', 0.2, 0.0, 0.97427866160920806)
('cosine', 0.2, 0.1, 0.97456440362138674)
('cosine', 0.2, 0.2, 0.98539382425615707)
('cosine', 0.2, 0.5, 1.0751417958149996)
('cosine', 0.2, 1.0, 1.3476818237388124)
('cosine', 0.4, 0.0, 0.97602871369402056)
('cosine', 0.4, 0.1, 0.97580397792675599)
('cosine', 0.4, 0.2, 0.98895492341641311)
('cosine', 0.4, 0.5, 1.0826321582875829)
('cosine', 0.4, 1.0, 1.3494705451767766)
('cosine', 0.6, 0.0, 0.9761838825072221)
('cosine', 0.6, 0.1, 0.97890604728191)
('cosine', 0.6, 0.2, 0.99196658616061817)
('cosine', 0.6, 0.5, 1.0842658092717246)
('cosine', 0.6, 1.0, 1.3509185073824312)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice.


('cosine', 0.8, 0.0, 0.98917766250886174)
('cosine', 0.8, 0.1, 0.99168909240677294)
('cosine', 0.8, 0.2, 1.0029189692136242)
('cosine', 0.8, 0.5, 1.0924770824809111)
('cosine', 0.8, 1.0, 1.3553654836142701)
('hamming', 0.2, 0.0, 0.98047346863069318)
('hamming', 0.2, 0.1, 0.98110097520562045)
('hamming', 0.2, 0.2, 0.9912506373156279)
('hamming', 0.2, 0.5, 1.0863198100440223)
('hamming', 0.2, 1.0, 1.3533256178897426)
('hamming', 0.4, 0.0, 0.98029102274205804)
('hamming', 0.4, 0.1, 0.98146045357185308)
('hamming', 0.4, 0.2, 0.99252719193452676)
('hamming', 0.4, 0.5, 1.0876014685731881)
('hamming', 0.4, 1.0, 1.351811701184003)
('hamming', 0.6, 0.0, 0.98204400590864682)
('hamming', 0.6, 0.1, 0.98424387301623384)
('hamming', 0.6, 0.2, 0.99697047810155426)
('hamming', 0.6, 0.5, 1.0889529431267735)
('hamming', 0.6, 1.0, 1.3525186959537132)
('hamming', 0.8, 0.0, 0.98780407129400738)
('hamming', 0.8, 0.1, 0.99145247563457528)
('hamming', 0.8, 0.2, 1.0028417448106515)
('hamming', 0.8, 0.5, 1.0921

In [ ]:
genre_matrix_completion = {}
for p in p_values:
    err = test_genre_based_matrix_completion(true_ratings, p=p, top_k=3)
    genre_matrix_completion[p] = err

print(genre_matrix_completion)

{0.8: 1.1598655995077276, 0.6: 1.1604394889038228, 0.4: 1.1602779989754564, 0.2: 1.1614843641175545}
